# Intro to PyWarp

This notebook demonstrates how to compute a metric, evaluate its energy tensor, and visualise the result with `plotly`.

In [ ]:
from warp.core import alcubierre_metric
from warp.core import energy_tensor
import plotly.graph_objects as go
import numpy as np

In [ ]:
# define grid and warp parameters
metric = alcubierre_metric(
    [3, 4, 4, 4],       # grid size (t, x, y, z)
    [1, 2, 2, 2],       # center of the warp bubble
    0.1,                # velocity fraction of c
    1.0,                # bubble radius
    1.0,                # bubble thickness
    [1, 1, 1, 1])       # scaling of coordinates
energy = energy_tensor(metric)

In [ ]:
# Extract the energy density (T_00 component)
energy_density = energy['tensor'][0,0]
print(energy_density.shape)

In [ ]:
# Create coordinate grids for visualisation
x = np.arange(energy_density.shape[1])
y = np.arange(energy_density.shape[2])
X, Y = np.meshgrid(x, y, indexing='ij')
# Use the middle time slice
Z = energy_density[1]
fig = go.Figure(data=go.Surface(x=X, y=Y, z=Z))
fig.update_layout(title='Energy Density', autosize=False,
                  width=600, height=600,
                  scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='T_00'))
fig.show()